In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
# plotting method

def plot_tdf_(df_, tdf_, side_: int = 0, liq_thr=0.1):
    plt.clf()
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type == 'entry']
        lc = longs[longs.type == 'close']
        ls = longs[longs.type == 'stop_loss']
        ls.price.plot(style='gx')
        le.price.plot(style='b.')
        longs.pos_price.plot(style='b--')
        if 'close_price' in longs.columns:
            longs.close_price.plot(style='r--')
        lc.price.plot(style='ro')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type == 'entry']
        sc = shrts[shrts.type == 'close']
        ss = shrts[shrts.type == 'stop_loss']
        ss.price.plot(style='gx')
        se.price.plot(style='r.')
        shrts.pos_price.plot(style='r--')
        if 'close_price' in shrts.columns:
            shrts.close_price.plot(style='b--')
        sc.price.plot(style='bo')
    if 'liq_price' in tdf_.columns:
        tdf_.liq_price.where((tdf_.price - tdf_.liq_price).abs() / tdf_.price < liq_thr, np.nan).plot(style='k--')
    return plt


In [ ]:
exchange = 'bybit'
symbol = 'ETHUSD'
backtesting_results_dir = f'backtesting_results/{exchange}/{symbol}/'
print('sessions')
sessions = sorted(os.listdir(backtesting_results_dir))
sessions

In [ ]:
# select session, default is last of sorted sessions list
session_name = sessions[-1]
#session_name = 'session_001'
print('session', session_name)
session_dir = f"{backtesting_results_dir}{session_name}/"
results_filepath = f"{session_dir}results.txt"
backtesting_settings = json.load(open(session_dir + 'backtesting_settings.json'))
with open(results_filepath) as f:
    results = [json.loads(line) for line in f.readlines()]
rdf = pd.DataFrame(results)
rdfs = rdf.sort_values('gain', ascending=False)
print('\nn completed backtests', len(rdf))
rdfs.head(20)

In [ ]:
n_days = backtesting_settings['n_days']
trades_list_cache_filepath = f"{session_dir}{n_days}_days_trades_list_cache.npy"
print('loading trades list...')
trades_list = np.load(trades_list_cache_filepath, allow_pickle=True)
print('done')
print('making dataframe...')
df = pd.DataFrame(list(trades_list))
print('done')

In [ ]:
best = json.load(open(session_dir + 'best.json'))
ranges = json.load(open(session_dir + 'ranges.json'))
best = {k_: best[k_] for k_ in sorted(ranges)}
live_settings = json.load(open(session_dir + 'best_result_live_settings.json'))

# choose backtest to view.
# idx = 0 is first, idx = 1 is second, etc
idx = 0
backtest_result = dict(rdfs.iloc[idx])
tdf = pd.read_csv(f"{session_dir}backtest_trades/{backtest_result['key']}.csv").set_index('trade_id')

best_dict = dict(rdfs.iloc[idx])
for k in sorted(best_dict):
    if k in backtesting_settings:
        backtesting_settings[k] = best_dict[k]
    if k in live_settings:
        try:
            live_settings[k] = float(best_dict[k])
        except:
            live_settings[k] = best_dict[k]
        del best_dict[k]
    else:
        best_dict[k] = float(best_dict[k])
print('live settings, selected candidate')
print(json.dumps(live_settings, indent=4))

In [ ]:
print('price with bid ask entry thresholds')
ema = df.price.ewm(span=live_settings['indicator_settings']['tick_ema']['span'], adjust=False).mean()
bids_ = ema * (1 - live_settings['indicator_settings']['tick_ema']['spread'])
asks_ = ema * (1 + live_settings['indicator_settings']['tick_ema']['spread'])

df.price.iloc[::100].plot()
bids_.iloc[::100].plot()
asks_.iloc[::100].plot()

In [ ]:
# analyze results
longs = tdf[tdf.side == 'long']
shrts = tdf[tdf.side == 'shrt']
le = longs[longs.type == 'entry']
lc = longs[longs.type == 'close']
se = shrts[shrts.type == 'entry']
sc = shrts[shrts.type == 'close']

def gain_conv(x):
    return x * 100 - 100

biggest_pos_size = tdf.pos_size.abs().max()
pnl_sum = tdf.pnl.sum()
stop_loss_closes = tdf[tdf.type == 'stop_loss']
loss_sum = stop_loss_closes.pnl.sum()
gain = (backtesting_settings['starting_balance'] + pnl_sum) / backtesting_settings['starting_balance']
closest_liq = ((tdf.price - tdf.liq_price).abs() / tdf.price).min()
n_stop_loss = len(stop_loss_closes)
n_days = backtesting_settings['n_days']
average_daily_gain = gain ** (1 / n_days)
closes = tdf[tdf.type == 'close']
lines = []
lines.append(f'net pnl {pnl_sum:.6f}')
lines.append(f'loss sum {loss_sum:.6f}')
lines.append(f'gain {gain * 100 - 100:.2f}%')
lines.append(f'n_days {n_days}')
lines.append(f'average_daily_gain {(average_daily_gain - 1) * 100:.2f}%')
lines.append(f'n trades {len(tdf)}')
lines.append(f'n closes {len(closes)}')
lines.append(f'n stop loss closes {n_stop_loss}')
lines.append(f'biggest_pos_size {biggest_pos_size:.10f}')
lines.append(f'closest liq {closest_liq * 100:.4f}%')
lines.append(f"starting balance {backtesting_settings['starting_balance']}")
lines.append(f"long: {backtesting_settings['do_long']}, short: {backtesting_settings['do_shrt']}")

with open(f'{session_dir}backtest_result.txt', 'w') as f:
    for line in lines:
        print(line)
        f.write(line + '\n')


In [ ]:
# plots are saved in backtesting_results/{exchange}/{symbol}/{session_name}/
n_parts = 7
for z in range(n_parts):
    start_ = z / n_parts
    end_ = (z + 1) / n_parts
    print(start_, end_)
    fig = plot_tdf_(df, tdf.iloc[int(len(tdf) * start_):int(len(tdf) * end_)], liq_thr=0.1)
    fig.savefig(f'{session_dir}backtest_{z + 1}of{n_parts}.png')
fig = plot_tdf_(df, tdf, liq_thr=0.1)
fig.savefig(f'{session_dir}whole_backtest.png')


In [ ]:
counter = 0
idxs = []
for row in tdf.itertuples():
    if row.type == 'stop_loss':
        counter += 1
    else:
        if counter > 0:
            idxs.append(row.Index)
        counter = 0
pnlcumsum = tdf.pnl.cumsum()
plt.clf()
pnlcumsum.plot()
if idxs:
    pnlcumsum.loc[idxs].plot(style='ro')
plt.savefig(f'{session_dir}pnlcumsum_plot.png')

In [ ]:
plt.clf()
tdf.pos_size.plot()
plt.savefig(f'{session_dir}pos_sizes_plot.png')

In [ ]:
adg_ = tdf.average_daily_gain
print('min max', adg_.min(), adg_.max())
adg_.index = tdf.progress
plt.clf()
adg_.iloc[int(len(tdf) * 0.1):].plot()
plt.savefig(f'{session_dir}average_daily_gain_plot.png')

In [ ]:
# visualize behavior
# execute below cell repeatedly (up arrow, shift enter) to see backtest chunk by chunk
# adjust step to set zoom level
step = 240
i = -step

In [ ]:
i += step
tdfc = tdf.iloc[i:i+step]
plot_tdf_(df, tdf.iloc[i:i+step], liq_thr=0.01)

In [ ]:
tdfcj = tdfc.join(pd.Series(tdfc.price.diff(), name='price_diff'))
tdfcj.head(60)

In [ ]:
tdfcj.tail(60)

In [ ]:
closest_liqs = ((tdf.liq_price - tdf.price).abs() / tdf.price).sort_values()
closest_liqs.head()

In [ ]:
i = 0
iloc_ = tdf.index.get_loc(closest_liqs.index[i])
iminus = 20
iplus = 150
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc, liq_thr=0.1)

In [ ]:
tdfc.head(60)